In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# Running time of each algorithm (in milliseconds)
alg_times = np.zeros((14,1))
alg_times[0] = 163
alg_times[1] = 163
alg_times[2] = 190
alg_times[3] = 190
alg_times[4] = 206
alg_times[5] = 206
alg_times[6] = 120
alg_times[7] = 120
alg_times[8] = 83
alg_times[9] = 83
alg_times[10] = 83
alg_times[11] = 83
alg_times[12] = 170
alg_times[13] = 170

# Time constraint: The total duration of the algorithms cannot exceed 600 milliseconds
alg_time_thr = 600


# Compute the total computational time for the fusion algorithm
def compute_total_time(M):
    is_used = np.zeros((14,1))
    for i in range(15):
        for j in range(15):
            if(M[i,j] != 0):
                if(i>=1):
                    is_used[i-1] = 1
                if(j>=1):
                    is_used[j-1] = 1

    total_dur = np.dot(is_used.T,alg_times)
    return total_dur[0,0]

# Evaluation metric
def compute_eval(fused_score):
    look_at_FAR = 0.0001
    # calculating FAR and FRR
    sort = np.argsort(fused_score[:,1])

    #sort = np.concatenate([sort[-2:],sort[:-2]], axis=0)
    scores = fused_score[sort]
    totpos = sum(scores[:,0])
    totneg = scores.shape[0]-totpos
    fa = (np.cumsum(scores[:,0]-1)+totneg)/totneg
    fr = np.cumsum(scores[:,0])/totpos

    i=0
    while fa[i]>look_at_FAR:
        i+=1

    return scores[i][1], fa[i], fr[i]


In [2]:
train_fname = 'train15_telecom.txt'
train_data = np.loadtxt(train_fname, dtype=np.float) #The first column contains the labels, the rest of the columns contains the scores

# Extract the labels
y_trn = train_data[:,0].astype(int)

# Extract the score vectors
s_trn = train_data.copy()
# Put a 1 in front of all the scores (see the "Fusion method" section above)
s_trn[:,0] = 1;

In [34]:
from itertools import product
train_fname = 'train15_telecom.txt'
train_data = np.loadtxt(train_fname, dtype=np.float)
y = train_data[:,0]
S = train_data[:,1:]

a = [[True, False], [True, False], [True, False], [True, False], [True, False], [True, False], [True, False], [
  True, False], [True, False], [True, False], [True, False], [True, False], [True, False], [True, False]]

pd_analysis_frr = pd.DataFrame(columns=['Classifier', 'Threshold', 'FAR', 'FRR'])

for combo in product(*a):
  total = np.sum(alg_times[np.array(combo)])
  if total < alg_time_thr:
      x = np.sum(S[:, np.array(combo)], axis=1)
      fuse = np.c_[y, x]
      thr, fa, fr = compute_eval(fuse)
      row = {'Classifier': np.argwhere(np.array(combo)==1), 'Threshold': thr, 'FAR': fa, 'FRR': fr}
      pd_analysis_frr = pd_analysis_frr.append(row, ignore_index=True)
     
pd_analysis_frr.sort_values(by=['FRR'], ascending=[True])
pd_analysis_frr.to_csv('classementfrr.txt', sep='\t')

In [ ]:
pd.apipipipina

In [3]:
train = pd.DataFrame(train_data)
train = train.replace([np.inf, -np.inf], np.nan)
train = train.dropna()
train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.0,4056.64,4329.06,4258.26,4487.92,4438.95,4624.01,4295.81,4516.67,5566.92,5365.55,3534.17,3877.52,4933.86,4970.20
1,0.0,2284.19,2448.09,2148.12,2246.95,2379.43,2581.86,2530.12,2782.97,2397.09,2606.08,1906.96,1856.95,2743.85,3048.56
2,0.0,2158.96,2263.43,2369.36,2567.98,2385.78,2590.60,2320.31,2499.47,2155.66,2258.41,1938.28,1910.30,2837.74,3158.75
3,0.0,2257.80,2410.03,2297.49,2467.10,2280.85,2443.30,2293.67,2461.65,2200.40,2325.69,1989.81,1996.23,2548.51,2806.69
4,0.0,2094.92,2164.81,1830.91,1723.67,2239.55,2383.45,2038.94,2076.11,2465.04,2697.64,1617.28,1317.34,2802.02,3117.27


In [4]:
array = train.values
X = array[:,1:15]
y = array[:,0]

In [125]:
train.describe()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
count,2.048840e+06,2.048840e+06,2.048840e+06,2.048840e+06,2.048840e+06,2.048840e+06,2.048840e+06,2.048840e+06,2.048840e+06,2.048840e+06,2.048840e+06,2.048840e+06,2.048840e+06,2.048840e+06,2.048840e+06
mean,2.492532e-02,2.573612e+03,2.733487e+03,2.569328e+03,2.727719e+03,2.637301e+03,2.829041e+03,2.664980e+03,2.863117e+03,2.472303e+03,2.610541e+03,2.310921e+03,2.378375e+03,2.638927e+03,2.817327e+03
std,1.558976e-01,7.418988e+02,7.974593e+02,7.693139e+02,7.846836e+02,6.980129e+02,7.413406e+02,7.468091e+02,7.142936e+02,7.094361e+02,7.465448e+02,7.412774e+02,7.689539e+02,8.320946e+02,7.518738e+02
min,0.000000e+00,8.004000e+01,-8.527270e+03,1.802900e+02,-5.867700e+03,5.351000e+01,-9.846110e+03,1.002070e+03,-2.503200e+02,8.172000e+02,-9.182300e+02,1.244010e+03,4.579500e+02,9.310700e+02,-4.910000e+02
25%,0.000000e+00,2.138500e+03,2.232240e+03,2.149470e+03,2.249000e+03,2.229640e+03,2.368927e+03,2.265550e+03,2.421240e+03,2.072810e+03,2.130070e+03,1.921370e+03,1.881610e+03,2.227810e+03,2.366240e+03
50%,0.000000e+00,2.386910e+03,2.592150e+03,2.392620e+03,2.599970e+03,2.460330e+03,2.691370e+03,2.490080e+03,2.730730e+03,2.325930e+03,2.507390e+03,2.142370e+03,2.238160e+03,2.450320e+03,2.678010e+03
75%,0.000000e+00,2.772650e+03,3.082760e+03,2.747730e+03,3.053190e+03,2.823080e+03,3.141780e+03,2.827520e+03,3.146940e+03,2.670540e+03,2.959863e+03,2.457520e+03,2.687620e+03,2.775990e+03,3.086690e+03
max,1.000000e+00,7.917510e+03,6.519120e+03,9.308740e+03,7.049270e+03,7.830500e+03,6.482930e+03,9.475980e+03,7.107590e+03,8.637100e+03,6.804020e+03,1.027537e+04,7.372830e+03,1.118469e+04,7.650540e+03


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
forest = ExtraTreesClassifier(n_estimators=250,
                              random_state=0, oob_score=True)

forest.fit(X, y)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

## AdaBoost + polynomialFeatures sur 8 9 10 11 13

In [5]:
X_ex = np.array([X[:,6],X[:,8],X[:,9], X[:,10], X[:,12]]).reshape(2048840,5)
X_ex.shape

(2048840, 5)

In [6]:
from sklearn.preprocessing import PolynomialFeatures
model = PolynomialFeatures(degree=2)
X_poly = model.fit_transform(X_ex)
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators=150)
ada.fit(X_poly, y)
print(ada.feature_importances_)

[ 0.          0.01333333  0.04        0.06        0.05333333  0.04666667
  0.02666667  0.06666667  0.05333333  0.07333333  0.04666667  0.05333333
  0.04666667  0.04666667  0.12        0.03333333  0.06        0.04
  0.03333333  0.06666667  0.02      ]


In [20]:
X_ex = np.array([X[:,8],X[:,9],X[:,10], X[:,13]]).reshape(2048840,4)
from sklearn import model_selection
from sklearn.ensemble import ExtraTreesClassifier
extra = ExtraTreesClassifier(n_estimators=150)
extra.fit(X_ex, y)
print(extra.feature_importances_)

[ 0.24976651  0.24977909  0.25022279  0.25023161]


In [43]:
#Prepare a fusion matrix
M = np.zeros((15,15))
#M = np.random.rand(15,15)
#Example: the matrix will only average the first and the third algorithms:

M[0, 9] = 0.92
M[0, 10] = 1
M[0, 12] = 1
M[0,13] =1





 
#Example: Make the fusion for the first image pair:
#cur_s = s_trn[0]
#cur_s_hat = np.dot(cur_s.T,np.dot(M,cur_s)) 

#Check if the time constraint is satisfied:
tot_dur = compute_total_time(M)
print (tot_dur)

if(tot_dur <= alg_time_thr):
    print ("The total running time of the fusion is acceptable!")
else:
    print ("The total running time of the fusion is NOT acceptable!")


419.0
The total running time of the fusion is acceptable!


In [44]:
#Evaluation
look_at_FAR = 0.0001
#apply fusion on scores  
fuse = np.multiply(s_trn[:,None,:]*s_trn[:,:,None], M)
fuse = np.concatenate([np.reshape(y_trn, [-1,1]), np.reshape(np.sum(fuse, axis=(1,2)), [-1,1])], axis=1)
fuse[np.isnan(fuse)]=-float("inf")

#compute the FRR at FAR = 0.01%
thr, fa, fr = compute_eval(fuse)

print ("Score at FAR="+str(look_at_FAR*100.0)+"%")
print ("threshold :", thr, "far :",fa, "frr :", fr)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in multiply
  after removing the cwd from sys.path.


Score at FAR=0.01%
threshold : 17576.3924 far : 9.9610468009e-05 frr : 0.0894832683911


In [45]:
#Submission

#Write the matrix M to the disk:
np.savetxt('M_pred.txt', M, fmt='%f')